In [2]:
# Import dependencies

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.simplefilter('ignore')

In [3]:
# The original flight delay data
delays_df = pd.read_csv("US-Airlines-delayfile-2018.csv")

In [17]:
# Separate the data by quarters as the entire dataset will not process due to the dataframes hogging all available memory.

delays_df = delays_df.loc[(delays_df["FL_DATE"] <= "2018-03-31") & (delays_df["FL_DATE"] >= "2018-01-01"),:]
delays_df = delays_df.loc[(delays_df["FL_DATE"] <= "2018-06-30") & (delays_df["FL_DATE"] >= "2018-04-01"),:]
delays_df = delays_df.loc[(delays_df["FL_DATE"] <= "2018-09-30") & (delays_df["FL_DATE"] >= "2018-07-01"),:]
delays_df = delays_df.loc[(delays_df["FL_DATE"] <= "2018-12-31") & (delays_df["FL_DATE"] >= "2018-10-01"),:]

In [31]:
# Airport names, city, state, country, latitude, longitude corresponding to DEST & ORIGIN code data
airports = pd.read_csv("airports.csv")

In [32]:
# Airline names corresponding to OP_CARRIER code data
airlines = pd.read_csv("Airline_codes.csv")

In [33]:
# Fill all NAN with zeros
delays_df=delays_df.fillna(0)

In [34]:
# Drop these two columns as they are mostly useless
delays_df=delays_df.drop(["CANCELLATION_CODE", "Unnamed: 27"], axis=1)

In [35]:
# Get Date and Month columns
delays_df['FL_DATE'] = pd.to_datetime(delays_df['FL_DATE'])
delays_df["DAY"] = delays_df['FL_DATE'].dt.day
delays_df["MONTH"] = delays_df['FL_DATE'].dt.month

In [36]:
# Merge the Carrier Codes for Airline Names
delays_df=pd.merge(delays_df, airlines, on='OP_CARRIER')

In [37]:
# Separate destination and origin flight names
airports_dest = airports.copy()
airports_origin = airports.copy()
airports_dest = airports_dest.drop([ \
                "ORIGIN", "ORIGIN_AIRPORT","ORIGIN_CITY", "ORIGIN_STATE", "ORIGIN_LATITUDE", "ORIGIN_LONGITUDE"], axis=1)



airports_origin = airports_origin.drop([ \
                "DEST", "DEST_AIRPORT", "DEST_CITY", "DEST_STATE", "DEST_LATITUDE", "DEST_LONGITUDE"], axis=1)

In [38]:
# Merge the airport variables using the Destination codes
delays_df = pd.merge(delays_df, airports_dest, on="DEST")

In [39]:
# Merge the airport variables using the Origin codes
delays_df = pd.merge(delays_df, airports_origin, on="ORIGIN")

In [40]:
# Check the length of the new dataframe
# Some data gets lost through the two merges but it is only around 1%.
len(delays_df)

1774250

In [41]:
# Dummy variables for flights arrival delay
delays_df['ARRIVAL_DELAY'] = 1*np.ravel(delays_df['ARR_DELAY']>0)

In [42]:
# Dummy variables for flight departure delays
delays_df['DEPARTURE_DELAY'] = 1*np.ravel(delays_df['DEP_DELAY']>0)

In [43]:
# Get the day of the week into its own column
from datetime import date

delays_df['FL_DATE'] = pd.to_datetime(delays_df['FL_DATE'])
delays_df["WEEKDAY"] = delays_df["FL_DATE"].dt.strftime('%A')

In [44]:
# Dictionaries for time of d

Time_of_day = {range(400, 1000): "Morning",
               range(1000, 1800): 'Afternoon', 
               range(1800, 2400): 'Night', 
               range(0, 400) : 'Night'}

Time_of_day_dummy = {range(400, 1000): 0,
               range(1000, 1800): 1, 
               range(1800, 2400): 2, 
               range(0, 400) : 2}

In [45]:
# Makes a departure time variable column using the time of day dictionary
delays_df['DEPARTURE_TIME_OF_DAY'] = delays_df['CRS_DEP_TIME'].apply(  \
                            lambda x: next((y for z, y in Time_of_day.items() if x in z), 0))

In [46]:
# Makes a departure dummy variable column using the time of day dictionary
delays_df['DEPARTURE_TIME_OF_DAY_DUMMY'] = delays_df['CRS_DEP_TIME'].apply(  \
                            lambda x: next((y for z, y in Time_of_day_dummy.items() if x in z), 0))

In [47]:
# Makes a arrival time variable column using the time of day dummy dictionary
delays_df['ARRIVAL_TIME_OF_DAY'] = delays_df['CRS_ARR_TIME'].apply(  \
                            lambda x: next((y for z, y in Time_of_day.items() if x in z), 0))

In [48]:
# Makes a arrival dummy variable column using the time of day dummy dictionary
delays_df["ARRIVAL_TIME_OF_DAY_DUMMY"] = delays_df['CRS_ARR_TIME'].apply(  \
                            lambda x: next((y for z, y in Time_of_day_dummy.items() if x in z), 0))

In [49]:
# Dummy variables for differing departure delay times
delays_df['DEPARTURE_DELAY_OVER_60_MINUTES'] = 1*np.ravel(delays_df["DEP_DELAY"] <= -60)
delays_df['DEPARTURE_DELAY_OVER_45_MINUTES'] = 1*np.ravel(delays_df["DEP_DELAY"] <= -45)
delays_df['DEPARTURE_DELAY_OVER_30_MINUTES'] = 1*np.ravel(delays_df["DEP_DELAY"] <= -30)
delays_df['DEPARTURE_DELAY_OVER_15_MINUTES'] = 1*np.ravel(delays_df["DEP_DELAY"] <= -15)

In [50]:
# Dummy variables for differing arrival delay times
delays_df['ARRIVAL_DELAY_OVER_60_MINUTES'] = 1*np.ravel(delays_df["ARR_DELAY"] <= -60)
delays_df['ARRIVAL_DELAY_OVER_45_MINUTES'] = 1*np.ravel(delays_df["ARR_DELAY"] <= -45)
delays_df['ARRIVAL_DELAY_OVER_30_MINUTES'] = 1*np.ravel(delays_df["ARR_DELAY"] <= -30)
delays_df['ARRIVAL_DELAY_OVER_15_MINUTES'] = 1*np.ravel(delays_df["ARR_DELAY"] <= -15)

In [51]:
# Dummy variables for flights of east coast origin/destination
delays_df['EAST_COAST_ORIGIN'] = 1*np.ravel(delays_df["ORIGIN_longitude"] >= -83)
delays_df['EAST_COAST_DEST'] = 1*np.ravel(delays_df["DEST_LONGITUDE"] >= -83)

In [52]:
# Dummy variables for flights of west coast origin/destination
delays_df['WEST_COAST_ORIGIN'] = 1*np.ravel(delays_df["ORIGIN_LATITUDE"] <= -114)
delays_df['WEST_COAST_DEST'] = 1*np.ravel(delays_df["DEST_LATITUDE"] <= -114)

In [53]:
# Make a new dataframe to have the columns ordered in a similar way to the original file
delays_df = delays_df[[ \
        "FL_DATE", "DAY", "MONTH", "WEEKDAY", "OP_CARRIER", "AIRLINE", "OP_CARRIER_FL_NUM", "ORIGIN", \
        "ORIGIN_AIRPORT", "ORIGIN_CITY", "ORIGIN_STATE", "ORIGIN_LATITUDE", "ORIGIN_LONGITUDE", "EAST_COAST_ORIGIN", \
        "WEST_COAST_ORIGIN", "CRS_DEP_TIME", "DEP_TIME", "DEPARTURE_TIME_OF_DAY", "DEPARTURE_TIME_OF_DAY_DUMMY", \
        "DEP_DELAY", "DEPARTURE_DELAY", "DEPARTURE_DELAY_OVER_15_MINUTES", "DEPARTURE_DELAY_OVER_30_MINUTES", \
        "DEPARTURE_DELAY_OVER_45_MINUTES", "DEPARTURE_DELAY_OVER_60_MINUTES", \
        "TAXI_OUT", "WHEELS_OFF", "AIR_TIME", "CRS_ELAPSED_TIME", "ACTUAL_ELAPSED_TIME", \
        "DISTANCE", "WHEELS_ON", "TAXI_IN", "DEST", \
        "DEST_AIRPORT", "DEST_CITY", "DEST_STATE", "DEST_LATITUDE", "DEST_LONGITUDE", "EAST_COAST_DEST",  \
        "WEST_COAST_DEST", "CRS_ARR_TIME", "ARR_TIME", "ARR_DELAY", "ARRIVAL_DELAY", "ARRIVAL_DELAY_OVER_15_MINUTES", \
        "ARRIVAL_DELAY_OVER_30_MINUTES", "ARRIVAL_DELAY_OVER_45_MINUTES", "ARRIVAL_DELAY_OVER_60_MINUTES", \
        "ARRIVAL_DELAY_OVER_60_MINUTES", "ARRIVAL_TIME_OF_DAY", "ARRIVAL_TIME_OF_DAY_DUMMY", "CANCELLED", "DIVERTED", \
        "CARRIER_DELAY", "WEATHER_DELAY", "NAS_DELAY", "SECURITY_DELAY", "LATE_AIRCRAFT_DELAY"]]

In [30]:
# This was to change all non-object columns to floats for ease of analysis purposes.
# However, memory issues prevented it from being used properly.

# cols = delays_df.select_dtypes(exclude=['object']).columns
# delays_df[cols] = delays_df[cols].apply(pd.to_numeric, downcast='float', errors='coerce')
# delays_df.dtypes

In [54]:
# Output new csv
delays_df.to_csv("Delay_fourth_quarter1.csv",index=False)